In [1]:
import os
import socket
from pathlib import Path
from PIL import Image
import timm
from sklearn.model_selection import GroupKFold

from datetime import datetime, timedelta
import pandas as pd
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import h5py

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2


# =========================================
# 📦 导入依赖
# =========================================
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from torch import nn
from torch.utils.data import Dataset, DataLoader


# =========================================
# ⚙️ 0️⃣ 全局参数配置
# =========================================
import torch
from torchvision import models
from torchvision.models import get_model_weights
from pathlib import Path
import os

from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 使用设备: {device}")

d:\Software\conda\envs\kaggle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 使用设备: cuda


In [ ]:
# 初始化
if socket.gethostname() == 'hao-2':
    dir = Path('D:/DATA_hao/Kaggle_/csiro-biomass/')
    DIRS = {
    "dir":        dir,                                       
    "train":     Path(dir, "train"),                              
    "test":     Path(dir, "test"),                              
    "model":     Path(dir,"DualStream_multihead"),              
    "data":     Path(dir),   
    }
else:
    dir = Path('/kaggle/input/csiro-biomass')
    DIRS = {
    "dir":        dir,                                       
    "train":     Path(dir, "train"),                              
    "test":     Path(dir, "test"),                              
    "model":     Path('/kaggle/input', "dualstream-multihead-model"),              
    "data":     Path("/kaggle/working/"),   
    }





# # 自动创建目录
# for key, path in DIRS.items():
#     os.makedirs(path, exist_ok=True)

# 打印时一行一个地址
print("✅ 路径已创建：\n")
for key, path in DIRS.items():
    print(f"{key:<12} : {path}")






✅ 路径已创建：

dir          : D:\DATA_hao\Kaggle_\csiro-biomass
train        : D:\DATA_hao\Kaggle_\csiro-biomass\train
test         : D:\DATA_hao\Kaggle_\csiro-biomass\test
model        : D:\DATA_hao\Kaggle_\csiro-biomass\DualStream_multihead
data         : D:\DATA_hao\Kaggle_\csiro-biomass


In [ ]:
# 小函数
def show_df_info(df, name: str):
    """
    打印单个 DataFrame 的形状与列名信息。
    参数:
        df   : pandas.DataFrame
        name : 显示名称（字符串）
    """
    print(f"📊 {name:<16} shape: {str(df.shape):<16}  列名: {df.columns.tolist()}")



def move_column_first(df, col_name):
    """
    将 DataFrame 中指定列移动到最前面。
    参数:
        df (pd.DataFrame): 原始数据框
        col_name (str): 要移动到最前面的列名
    返回:
        pd.DataFrame: 调整后的新 DataFrame
    """
    if col_name not in df.columns:
        raise ValueError(f"列 '{col_name}' 不存在于 DataFrame 中。")

    cols = [col_name] + [c for c in df.columns if c != col_name]
    return df[cols]


from sklearn.metrics import r2_score

def weighted_r2(y_true_df, y_pred_df):
    weights = {
        "Dry_Green_g": 0.1,
        "Dry_Dead_g": 0.1,
        "Dry_Clover_g": 0.1,
        "GDM_g": 0.2,
        "Dry_Total_g": 0.5
    }

    r2_dict = {}
    for col in weights.keys():
        r2_dict[col] = r2_score(y_true_df[col], y_pred_df[col])

    weighted_score = sum(r2_dict[k] * w for k, w in weights.items())
    return weighted_score, r2_dict



In [ ]:
# 📘 数据读取与预处理

# 1️⃣ 读取原始数据
df_file_path = Path(DIRS["dir"] / "train.csv")
df = pd.read_csv(df_file_path)
show_df_info(df, "train.csv")

# 2️⃣ 提取唯一 ID（例如 "ID1011485656__Dry_Green_g" → "ID1011485656"）
df["ID"] = df["sample_id"].str.split("__").str[0]

# 3️⃣ 将 ID 列移动到最前面
df = move_column_first(df, "ID")
show_df_info(df, "df")

# 🧩 目标值透视（行转列）
df_targets = (
    df
    .pivot_table(
        index="ID",
        columns="target_name",
        values="target",
        aggfunc="first"
    )
    .reset_index()
)
df_targets.columns.name = None  # 去掉多级列名层次
show_df_info(df_targets, "df_targets")

# 🧬 提取元信息（每个 ID 仅保留一行）
meta_cols = [
    "ID", "image_path", "Sampling_Date", "State",
    "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"
]
df_meta = df[meta_cols].drop_duplicates(subset="ID")
show_df_info(df_meta, "df_meta")

# 🔗 合并元信息与目标数据
df_train = pd.merge(df_meta, df_targets, on="ID", how="left")

show_df_info(df_train, "df_train")


# df_wide.head()


In [ ]:
# 🧠 MyDualStreamModel：双流 + 多头回归 + 内部训练逻辑


class WeightedSmoothL1Loss(nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = list(weights.values())
        self.loss_fn = nn.SmoothL1Loss(reduction="none")

    def forward(self, pred, target):
        losses = self.loss_fn(pred, target)
        weighted = sum(losses[:, i] * w for i, w in enumerate(self.weights))
        return weighted.mean()





class MyDualStreamModel(nn.Module):
    def __init__(self, 
                backbone_name="convnext_tiny", 
                pretrained=True, 
                freeze_ratio=0.8,
                weights_dict=None):
        """
        参数:
        - backbone_name: timm 模型名称 (如 convnext_tiny, resnet50)
        - pretrained: 是否加载 ImageNet 权重
        - freeze_ratio: 冻结比例（0~1）
        - weights_dict: 各目标权重 (dict), 用于 WeightedSmoothL1Loss
        """
        super().__init__()

        # 1️⃣ Backbone
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
        in_dim = self.backbone.num_features

        # 2️⃣ 冻结部分参数
        params = list(self.backbone.parameters())
        freeze_until = int(len(params) * freeze_ratio)
        for i, p in enumerate(params):
            p.requires_grad = i >= freeze_until  # 前部分冻结，后部分可学习

        # 3️⃣ 双流融合
        self.fusion_dim = in_dim * 2

        # 4️⃣ 三个输出 Head
        def make_head():
            return nn.Sequential(
                nn.Linear(self.fusion_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )

        self.head_total = make_head()
        self.head_gdm   = make_head()
        self.head_green = make_head()

        # 5️⃣ 损失函数（Weighted SmoothL1Loss）
        self.loss_fn = WeightedSmoothL1Loss(weights_dict) if weights_dict else nn.SmoothL1Loss()



    # ------------------------------------------------------------
    # 🔁 Forward
    # ------------------------------------------------------------
    def forward(self, img_left, img_right):
        feat_left  = self.backbone(img_left)
        feat_right = self.backbone(img_right)
        fused = torch.cat([feat_left, feat_right], dim=1)

        total = self.head_total(fused)
        gdm   = self.head_gdm(fused)
        green = self.head_green(fused)
        preds = torch.cat([green, gdm, total], dim=1)
        return preds  # shape: [batch, 3]

    # ------------------------------------------------------------
    # 🧮 损失计算（内部调用）
    # ------------------------------------------------------------
    def compute_loss(self, preds, targets):
        return self.loss_fn(preds, targets)


In [ ]:
import numpy as np
from PIL import Image
from pathlib import Path
import torch
from torch.utils.data import Dataset

class DualStreamDataset(Dataset):
    def __init__(self, df, image_dir, target_cols=None, transform=None):
        """
        df: DataFrame，包含 image_path 列
        image_dir: 图像目录
        target_cols: 如果是训练集，指定目标列
        transform: Albumentations 变换
        """
        self.df = df
        self.image_dir = image_dir
        self.target_cols = target_cols
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(self.image_dir, str(row["image_path"]))
        
        # ====== 1️⃣ 安全加载 ======
        if not img_path.exists():
            print(f"⚠️ 图片不存在: {img_path}")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        else:
            try:
                image = Image.open(img_path).convert("RGB")
            except Exception as e:
                print(f"⚠️ 无法读取图片: {img_path} ({e})")
                image = np.zeros((1000, 2000, 3), dtype=np.uint8)

        # ====== 2️⃣ 确保转换为 NumPy 数组 ======
        image = np.array(image)  # 转换为 NumPy 数组
        h, w, _ = image.shape
        mid = w // 2
        
        # 拆分成左右两个 patch
        img_left = image[:, :mid]
        img_right = image[:, mid:]

        # ====== 4️⃣ 应用 Albumentations 变换 ======
        if self.transform:
            img_left = self.transform(image=img_left)["image"]
            img_right = self.transform(image=img_right)["image"]

        # ====== 5️⃣ 返回结果 ======
        if self.target_cols is not None:
            targets = torch.tensor(row[self.target_cols].astype(float).values, dtype=torch.float32)
            return img_left, img_right, targets
        else:
            return img_left, img_right


In [ ]:
# 🔧 Albumentations 变换
def get_train_transforms(size=768):
    return A.Compose([
        A.Resize(size, size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ColorJitter(p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def get_valid_transforms(size=768):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


In [ ]:
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import GroupKFold


def train_with_groupkfold(
    df_train,
    save_dir,
    model_target_cols,
    get_train_transforms,
    get_valid_transforms,
    weights,
    freeze_ratio=0.8,
    batch_size=32,
    epochs=50,
    lr=1e-4,
    device=None,
    n_splits=5,
):

    scaler = GradScaler()  # ✅ 初始化缩放器（用于FP16梯度稳定）

    # 定义分层器（5折交叉验证）
    gkf = GroupKFold(n_splits=n_splits)

    # 取第一折
    df = df_train.copy()
    groups = df["Sampling_Date"]

    # 🔹 用于保存每折训练/验证损失
    fold_train_losses = []
    fold_val_losses = []

    for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups=groups)):
        print(f"Fold {fold:3d}: Train={len(train_idx)}, Valid={len(val_idx)}")
        train_df = df.iloc[train_idx].reset_index(drop=True)
        valid_df = df.iloc[val_idx].reset_index(drop=True)
            
        train_dataset = DualStreamDataset(
            train_df, DIRS["dir"], model_target_cols, transform=get_train_transforms(768)
        )
        valid_dataset = DualStreamDataset(
            valid_df, DIRS["dir"], model_target_cols, transform=get_valid_transforms(768)
        )

        # ✅ 增加 pin_memory 提高主机→GPU 传输速度
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

        # ✅ 模型优化：channels_last 内存布局 + AMP 兼容
        model = MyDualStreamModel("convnext_tiny", pretrained=True, freeze_ratio=freeze_ratio, weights_dict=weights)
        model = model.to(device)
        model = model.to(memory_format=torch.channels_last)

        optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

        # 🔹 每折记录损失
        train_losses_per_epoch = []
        val_losses_per_epoch = []

        for epoch in range(epochs):
            # === 🔥 启用混合精度 ===
            model.train()
            running_train_loss = []

            # ✅ tqdm 实时显示
            for batch in tqdm(train_loader, desc=f"Epoch {epoch+1:3d}/{epochs}"):
                img_left, img_right, targets = batch
                img_left, img_right, targets = (
                    img_left.to(device, non_blocking=True),
                    img_right.to(device, non_blocking=True),
                    targets.to(device, non_blocking=True),
                )

                optimizer.zero_grad(set_to_none=True)  # ✅ 更高效清空梯度

                # ✅ AMP混合精度上下文
                with autocast():
                    preds = model(img_left, img_right)
                    loss = model.compute_loss(preds, targets)

                scaler.scale(loss).backward()
                # scaler.unscale_(optimizer)  # 可选：如果想加梯度裁剪，可在此解缩放
                scaler.step(optimizer)
                scaler.update()

                running_train_loss.append(loss.item())

            avg_train_loss = float(np.mean(running_train_loss))

            # === 🧊 验证阶段（不用 AMP）===
            model.eval()
            val_losses = []
            with torch.no_grad():
                for batch in valid_loader:
                    img_left, img_right, targets = batch
                    img_left, img_right, targets = (
                        img_left.to(device, non_blocking=True),
                        img_right.to(device, non_blocking=True),
                        targets.to(device, non_blocking=True),
                    )
                    preds = model(img_left, img_right)
                    val_loss = model.compute_loss(preds, targets).item()
                    val_losses.append(val_loss)

            avg_val_loss = float(np.mean(val_losses))

            print(f"Epoch {epoch+1:3d} | Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")

            train_losses_per_epoch.append(avg_train_loss)
            val_losses_per_epoch.append(avg_val_loss)

        # ✅ 保存每折的所有epoch损失
        fold_train_losses.append(train_losses_per_epoch)
        fold_val_losses.append(val_losses_per_epoch)




        save_path = save_dir / f"model_weights_fold_{fold}.pt"
        torch.save(model.state_dict(), save_path)

        print(f"✅ 完整模型已保存到: {save_path}")
        

    # 获取最大epoch数（有时不同fold可能长度不同）
    max_epochs = max(len(x) for x in fold_train_losses)
    df = pd.DataFrame({"Epoch": range(1, max_epochs + 1)})

    # 逐个fold填充
    for i, (train_list, val_list) in enumerate(zip(fold_train_losses, fold_val_losses), start=1):
        # 若某个fold长度不足，则用None填充
        pad_train = train_list + [None] * (max_epochs - len(train_list))
        pad_val = val_list + [None] * (max_epochs - len(val_list))
        
        df[f"Train_Loss_Fold{i}"] = pad_train
        df[f"Val_Loss_Fold{i}"] = pad_val

    # 保存到Excel
    output_path = Path(save_dir, "fold_losses_wide.xlsx")
    df.to_excel(output_path, index=False)
    print(f"✅ 每Fold的Train/Val Loss已保存为列：{output_path}")

    return fold_train_losses, fold_val_losses

In [ ]:
weights           = {
    "Dry_Green_g": 0.1,
    "GDM_g":        0.2,
    "Dry_Total_g":  0.5
}

model_target_cols = ["Dry_Green_g", "GDM_g", "Dry_Total_g"]
target_cols       = ["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]




config = {
    "epochs" : 2,
    "freeze_ratio" : 0.8,
    "batch_size" : 32,
    "lr" : 1e-4,
}


# ==========================
# 🚀 启动训练（GroupKFold）
# ==========================



if socket.gethostname() == 'hao-2':


    time_str = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    history_DIR = Path(DIRS['model'], time_str)
    os.makedirs(history_DIR, exist_ok=True)

    fold_train_losses, fold_val_losses = train_with_groupkfold(
        df_train             = df_train,
        save_dir             = history_DIR,
        model_target_cols    = model_target_cols,
        get_train_transforms = get_train_transforms,
        get_valid_transforms = get_valid_transforms,
        weights              = weights,
        freeze_ratio         = config["freeze_ratio"],
        batch_size           = config["batch_size"],
        epochs               = config["epochs"],
        lr                   = config["lr"],
        device               = device,
    )




In [ ]:
# 🧮 后处理函数（恢复 5 个目标）
def recover_all_targets(df_pred_3):
    df = df_pred_3.copy()
    df["Dry_Clover_g"] = np.maximum(0, df["GDM_g"] - df["Dry_Green_g"])
    df["Dry_Dead_g"] = np.maximum(0, df["Dry_Total_g"] - df["GDM_g"])
    return df[["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]]



In [ ]:
# 📘 数据读取与预处理

# 1️⃣ 读取原始数据
df_file_path = Path(DIRS["dir"] / "test.csv")
df = pd.read_csv(df_file_path)
show_df_info(df, "test.csv")

# 2️⃣ 提取唯一 ID（例如 "ID1011485656__Dry_Green_g" → "ID1011485656"）
df["ID"] = df["sample_id"].str.split("__").str[0]

# 3️⃣ 将 ID 列移动到最前面
df = move_column_first(df, "ID")

df["target"] = 0

show_df_info(df, "df")

# 🧩 目标值透视（行转列）
df_targets = (
    df
    .pivot_table(
        index="ID",
        columns="target_name",
        values="target",
        aggfunc="first"
    )
    .reset_index()
)
df_targets.columns.name = None  # 去掉多级列名层次
show_df_info(df_targets, "df_targets")

# 🧬 提取元信息（每个 ID 仅保留一行）
# meta_cols = [
#     "ID", "image_path", "Sampling_Date", "State",
#     "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"
# ]

meta_cols = [
    "ID", "image_path"
]
df_meta = df[meta_cols].drop_duplicates(subset="ID")
show_df_info(df_meta, "df_meta")

# 🔗 合并元信息与目标数据
df_test = pd.merge(df_meta, df_targets, on="ID", how="left")

show_df_info(df_test, "df_wide")


In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader


def predict_ensemble_df(df_test, transform, model, model_target_cols, model_dir, device, batch_size=32, img_size=768):

    model_dir = model_dir
    print(f"模型目录: {model_dir}")
    assert model_dir.exists(), f"❌ 模型目录不存在: {model_dir}"

    # 🔍 搜索所有 fold 模型
    model_paths = sorted(model_dir.glob("model_weights_fold*.pt"))
    if not model_paths:
        raise FileNotFoundError(f"❌ 未找到模型文件: {model_dir}/model_weights_fold*.pt")

    print(f"🔹 检测到 {len(model_paths)} 个模型:")
    for p in model_paths:
        print("   -", p.name)

    # 3️⃣ 构建测试数据集
    test_dataset = DualStreamDataset(
        df_test,
        image_dir=DIRS["dir"],
        target_cols=None,
        transform=transform   
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    # 存储每个fold的预测
    fold_preds = []

    for fold, model_path in enumerate(model_paths):
        # print(f"🚀 加载模型 {fold+1}/{len(model_paths)}: {model_path.name}")

        # 1️⃣ 加载模型结构
        model = model

        # 2️⃣ 加载权重
        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict)
        model.eval()

        # 3️⃣ 推理
        preds_list = []
        with torch.no_grad():
            for img_left, img_right in test_loader:
                img_left, img_right = img_left.to(device, non_blocking=True), img_right.to(device, non_blocking=True)
                preds = model(img_left, img_right)
                preds_list.append(preds.cpu().numpy())

        fold_pred = np.concatenate(preds_list, axis=0)
        fold_preds.append(fold_pred)

    # 4️⃣ 多模型平均
    preds_mean = np.mean(fold_preds, axis=0)
    df_pred3 = pd.DataFrame(preds_mean, columns=model_target_cols)





    # 恢复完整的 5 个目标列
    df_pred5 = recover_all_targets(df_pred3)
    show_df_info(df_pred5, "df_pred5 ")


    # 追加样本 ID 并调整列顺序
    df_pred5["ID"] = df_test["ID"]
    df_pred5 = df_pred5[["ID"] + target_cols]




    # 打印结果预览
    show_df_info(df_pred5, "final df_pred5")

    return df_pred5


In [ ]:
tta_transforms = {
    "base": A.Compose([
        A.Resize(768, 768),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "hflip": A.Compose([
        A.Resize(768, 768),
        A.HorizontalFlip(p=1),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "vflip": A.Compose([
        A.Resize(768, 768),
        A.VerticalFlip(p=1),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
}

In [ ]:
# 1️⃣ 加载模型结构
model = MyDualStreamModel("convnext_tiny", 
                          pretrained=False, 
                          freeze_ratio=config["freeze_ratio"], 
                          weights_dict=weights)
model = model.to(device)
model = model.to(memory_format=torch.channels_last)


model_dir = history_DIR
model_dir = history_DIR




tta_preds = []

for name, tform in tta_transforms.items():
    print(f"🚀 Running TTA: {name}")


    transform = tform
    df_pred5 = predict_ensemble_df(
        df_test=df_test,
        transform = transform,
        model = model,
        model_target_cols=model_target_cols,
        model_dir = model_dir,
        device=device,
    )
    
    print(df_pred5)
    tta_preds.append(df_pred5[target_cols].values)

print("\n")
print(tta_preds)
mean_preds = np.mean(tta_preds, axis=0)

print(mean_preds)
df_pred_final = df_pred5.copy()
df_pred_final[target_cols] = mean_preds
print(df_pred_final)






In [ ]:
# =========================================
# 📤 5️⃣ 生成 Kaggle 提交文件 submission.csv
# =========================================
# 按指定顺序展开
ordered_target_cols = [
    "Dry_Clover_g",  # 1️⃣
    "Dry_Dead_g",    # 2️⃣
    "Dry_Green_g",   # 3️⃣
    "Dry_Total_g",   # 4️⃣
    "GDM_g"          # 5️⃣
]

df_submit = (
    df_pred5
    .melt(id_vars="ID", value_vars=ordered_target_cols,
          var_name="target_name", value_name="target")
)

# 组合成 Kaggle 所需的 sample_id
df_submit["sample_id"] = df_submit["ID"] + "__" + df_submit["target_name"]

df_submit = move_column_first(df_submit, "target")
df_submit = move_column_first(df_submit, "sample_id")

# 只保留 Kaggle 要的两列
df_submit = df_submit[["sample_id", "target"]]
df_submit
# 按 sample_id 排序（可选）
# df_submit = df_submit.sort_values("sample_id").reset_index(drop=True)

# 保存文件
df_submit.to_csv("submission.csv", index=False)
print("✅ 已生成提交文件 submission.csv")
